In [4]:
import pandas as pd

df = pd.read_excel("test_02.xlsx")

print(df)

   Sheet1     remove all special characters and white spaces
0  Sheet2  shuffle the words then remove all special char...
1  Sheet3            separate every character by white space
2  Sheet4  shuffle the words then separate every characte...
3  Sheet5                                       word removal
4  Sheet6                                    word truncation
5  Sheet7                                           initials
6  Sheet8                     simulated names, no true match


In [5]:
df_1 = pd.read_excel("test_02.xlsx", sheet_name="Sheet1")
df_2 = pd.read_excel("test_02.xlsx", sheet_name="Sheet2")
df_3 = pd.read_excel("test_02.xlsx", sheet_name="Sheet3")
df_4 = pd.read_excel("test_02.xlsx", sheet_name="Sheet4")
df_5 = pd.read_excel("test_02.xlsx", sheet_name="Sheet5")
df_6 = pd.read_excel("test_02.xlsx", sheet_name="Sheet6")
df_7 = pd.read_excel("test_02.xlsx", sheet_name="Sheet7")
df_8 = pd.read_excel("test_02.xlsx", sheet_name="Sheet8")



In [6]:
import pandas as pd
import re
import wordninja
from rapidfuzz import process, fuzz
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import os

# -------------------------
# Step 1: 数据加载与清洗
# -------------------------

# 读取 primary 和 alternate 数据
primary_names = pd.read_csv('primary.csv')
alternate_names = pd.read_csv('alternate.csv', nrows=1000)

# 读取测试数据
df_4 = pd.read_excel("test_02.xlsx", sheet_name="Sheet4")
df_2 = pd.read_excel("test_02.xlsx", sheet_name="Sheet2")

# -------------------------
# Step 2: 分词 + 排序 处理函数
# -------------------------

def normalize_and_sort_words(name):
    if pd.isna(name):
        return ''
    clean = re.sub(r'[^a-zA-Z0-9]', '', name)  # 去除非字母数字
    segments = wordninja.split(clean.lower())  # 自动分词
    return ' '.join(sorted(segments))

# 添加处理列
df_4['PROCESSED_NAME'] = df_4['NAME'].apply(normalize_and_sort_words)
df_2['PROCESSED_NAME'] = df_2['NAME'].apply(normalize_and_sort_words)

primary_names['PROCESSED_NAME'] = primary_names['NAME'].apply(normalize_and_sort_words)
primary_name_list = primary_names['PROCESSED_NAME'].tolist()
primary_id_list = primary_names['ID'].tolist()

# -------------------------
# Step 3: 匹配函数
# -------------------------

def match_test_variant(test_row):
    test_id = test_row['ID']
    test_variant = test_row['PROCESSED_NAME']

    if not test_variant:  # 空字符串跳过
        return test_id, None, None, 0

    match_result = process.extractOne(
        test_variant, 
        primary_name_list,
        processor=None,
        scorer=fuzz.token_sort_ratio
    )

    if match_result:
        match, score, _ = match_result
        try:
            matched_index = primary_name_list.index(match)
            matched_id = primary_id_list[matched_index]
        except ValueError:
            matched_id = None
    else:
        match, score, matched_id = None, 0, None

    return test_id, matched_id, match, score

# -------------------------
# Step 4: 并行匹配执行
# -------------------------

def fuzzy_match_test_to_primary(test_df):
    results = []
    correct_matches = 0
    total = len(test_df)

    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [
            executor.submit(match_test_variant, row)
            for row in test_df.to_dict('records')
        ]

        for future in tqdm(as_completed(futures), total=total, desc="匹配进度"):
            test_id, matched_id, matched_name, score = future.result()

            # 用当前的 test_df 获取原始行信息（而不是 df_4）
            original_row = test_df.loc[test_df['ID'] == test_id]
            original_name = original_row['NAME'].values[0] if not original_row.empty else ''
            original_processed = original_row['PROCESSED_NAME'].values[0] if not original_row.empty else ''

            is_correct = (test_id == matched_id)
            if is_correct and score < 75:
                print(f"低分但正确：{test_id} 得分: {score}")
            if is_correct:
                correct_matches += 1

            results.append({
                'ID': test_id,
                'NAME': original_name,
                'PROCESSED_NAME': original_processed,
                'MATCHED_ID': matched_id,
                'MATCHED_NAME': matched_name,
                'SCORE': score,
                'CORRECT': is_correct
            })

    accuracy = correct_matches / total if total else 0
    return pd.DataFrame(results), accuracy

# -------------------------
# Step 5: 执行并保存结果
# -------------------------

# 确保保存目录存在
os.makedirs("./Matched_results", exist_ok=True)

matched_df_4, acc_4 = fuzzy_match_test_to_primary(df_4)
matched_df_2, acc_2 = fuzzy_match_test_to_primary(df_2)

# 保存结果
matched_df_2.to_csv("./Matched_results/Test2_matched_results_df2.csv", index=False)
print(f"\ndf_2 准确率: {acc_2 * 100:.2f}%")
print("匹配结果已保存至 Test2_matched_results_df2.csv")

matched_df_4.to_csv("./Matched_results/Test2_matched_results_df4.csv", index=False)
print(f"\ndf_4 准确率: {acc_4 * 100:.2f}%")
print("匹配结果已保存至 Test2_matched_results_df4.csv")


匹配进度:   7%|▋         | 73/1000 [00:02<00:45, 20.35it/s] 

低分但正确：21575 得分: 69.23076923076923


匹配进度: 100%|██████████| 1000/1000 [00:03<00:00, 260.09it/s]

低分但正确：27163 得分: 74.35897435897436
低分但正确：37006 得分: 60.71428571428572
低分但正确：47333 得分: 73.13432835820896
低分但正确：23065 得分: 74.07407407407408
低分但正确：10999 得分: 72.88135593220339
低分但正确：8094 得分: 74.4186046511628
低分但正确：30887 得分: 70.96774193548387
低分但正确：49163 得分: 61.53846153846154
低分但正确：12274 得分: 68.18181818181819
低分但正确：10640 得分: 62.5
低分但正确：10601 得分: 73.91304347826086
低分但正确：50918 得分: 74.33628318584071
低分但正确：20257 得分: 68.18181818181819
低分但正确：7184 得分: 73.33333333333334
低分但正确：49267 得分: 74.68354430379746
低分但正确：51275 得分: 71.15384615384616
低分但正确：19809 得分: 65.95744680851064
低分但正确：11000 得分: 69.0909090909091
低分但正确：15342 得分: 72.0
低分但正确：37578 得分: 57.57575757575757
低分但正确：23175 得分: 69.44444444444444
低分但正确：41665 得分: 73.01587301587303
低分但正确：52202 得分: 61.08374384236453
低分但正确：28573 得分: 67.85714285714286
低分但正确：15674 得分: 73.97260273972603
低分但正确：11895 得分: 68.08510638297872
低分但正确：48104 得分: 72.59259259259258
低分但正确：44444 得分: 65.625
低分但正确：51475 得分: 74.19354838709677
低分但正确：8407 得分: 70.0
低分但正确：27164 得分: 69.38775510204081
低


匹配进度:  49%|████▊     | 487/1000 [00:00<00:00, 3594.91it/s]

低分但正确：50159 得分: 73.1958762886598
低分但正确：18095 得分: 69.38775510204081
低分但正确：46923 得分: 72.22222222222221
低分但正确：41277 得分: 74.35897435897436
低分但正确：16990 得分: 67.74193548387098
低分但正确：10849 得分: 70.0
低分但正确：8332 得分: 61.53846153846154
低分但正确：13407 得分: 64.86486486486487
低分但正确：24563 得分: 70.76923076923076
低分但正确：16476 得分: 73.07692307692308
低分但正确：15342 得分: 72.97297297297297
低分但正确：45741 得分: 67.45562130177515
低分但正确：11773 得分: 73.6842105263158
低分但正确：7286 得分: 64.28571428571428
低分但正确：8407 得分: 70.0
低分但正确：10359 得分: 74.57627118644068
低分但正确：50980 得分: 73.2824427480916
低分但正确：49795 得分: 72.34042553191489
低分但正确：29023 得分: 68.08510638297872
低分但正确：7585 得分: 63.01369863013699
低分但正确：52249 得分: 71.69811320754718


匹配进度:  85%|████████▍ | 847/1000 [00:02<00:00, 337.93it/s] 

低分但正确：48742 得分: 72.3076923076923
低分但正确：40942 得分: 74.4186046511628
低分但正确：21114 得分: 69.23076923076923
低分但正确：10640 得分: 62.5
低分但正确：52689 得分: 72.26890756302521
低分但正确：44787 得分: 74.32432432432432
低分但正确：41178 得分: 59.01639344262295
低分但正确：18087 得分: 70.27027027027026
低分但正确：48836 得分: 68.35443037974683
低分但正确：47548 得分: 70.89947089947091


匹配进度: 100%|██████████| 1000/1000 [00:02<00:00, 341.37it/s]

低分但正确：11909 得分: 64.28571428571428
低分但正确：27446 得分: 74.07407407407408
低分但正确：34916 得分: 62.96296296296296
低分但正确：10657 得分: 67.64705882352942

df_2 准确率: 96.10%
匹配结果已保存至 Test2_matched_results_df2.csv

df_4 准确率: 97.20%
匹配结果已保存至 Test2_matched_results_df4.csv


In [6]:
from locale import D_FMT
import pandas as pd
from rapidfuzz import process, fuzz
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm 
import re
import wordninja # 导入 tqdm

# 加载数据
alternate_names = pd.read_csv('alternate.csv', nrows=1000)
primary_names = pd.read_csv('primary.csv')
test_names = df_1

# 查看数据
print(alternate_names.head())
print(primary_names.head())
print(test_names.head())

# 将 primary_names 的 NAME 字段转换为列表，方便快速匹配
primary_names_list = primary_names['NAME'].tolist()
primary_ids = primary_names['ID'].tolist()

def normalize_and_sort_words(name):
    clean = re.sub(r'[^a-zA-Z0-9]', '', name)
    # Step 2: 小写（便于 wordninja 识别）再切分
    segments = wordninja.split(clean.lower())
    # Step 3: 恢复大写首字母（可选）
    return ' '.join(segments)

def recover_and_segment(name):
    if pd.isna(name):  # 处理缺失值
        return ''
    clean = re.sub(r'[^a-zA-Z0-9]', '', name)  # 去掉所有非字母数字字符
    segments = wordninja.split(clean.lower())  # 分词
    return ' '.join([w.upper() for w in segments])  # 全大写输出

# 匹配函数
def match_test_variant(test_row):
    test_id = test_row['ID']
    test_variant_split = recover_and_segment(test_row['NAME'])
    test_variant = test_row['NAME']

    
    # 使用 rapidfuzz 找到最匹配的 NAME
    match1, score1, _ = process.extractOne(
        test_variant, 
        primary_names_list,
        processor=lambda x: re.sub(r'[^a-z0-9]', '', x.lower()),
        scorer=fuzz.WRatio  # 使用 token_sort_ratio 提高匹配精度
    )

    match2, score2, _ = process.extractOne(
        test_variant_split, 
        primary_names_list,
        processor=lambda x: re.sub(r'[^a-z0-9 ]', '', x.lower()),
        scorer=fuzz.token_sort_ratio  # 使用 token_sort_ratio 提高匹配精度
    )

    if score1 > score2:
        match = match1
        score = score1
    else:
        match = match2
        score = score2
    # 获取匹配的 NAME 对应的 ID
    matched_index = primary_names_list.index(match)
    matched_id = primary_ids[matched_index]

    # 返回匹配结果
    return {
        'test_id': test_id,
        'test_name': test_variant,
        'matched_id': matched_id,
        'matched_name': match,
        'score': score,
        'is_correct': int(test_id == matched_id and score > 75)
    }

# 并行匹配
# def fuzzy_match_test_to_primary(test_names, primary_names):
    correct_matches = 0  # 记录正确匹配的数量
    total_tests = len(test_names)  # 总测试数量

    # 使用多线程并行匹配
    with ThreadPoolExecutor() as executor:
        # 提交任务
        futures = [
            executor.submit(match_test_variant, test_row)
            for test_row in test_names.to_dict('records')
        ]

        # 使用 tqdm 包装 futures，显示进度条
        for future in tqdm(as_completed(futures), total=total_tests, desc="匹配进度"):
            test_id, matched_id, score = future.result()
            if test_id == matched_id:
                if score > 80:
                    correct_matches += 1

    # 计算准确率
    accuracy = correct_matches / total_tests
    return accuracy

def fuzzy_match_test_to_primary(test_names, primary_names, file_prefix='df'):
    total_tests = len(test_names)
    matched_results = []

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(match_test_variant, row) for row in test_names.to_dict('records')]
        for future in tqdm(as_completed(futures), total=total_tests, desc=f"匹配中 {file_prefix}"):
            result = future.result()
            matched_results.append(result)

    # 转换为 DataFrame
    results_df = pd.DataFrame(matched_results)
    accuracy = results_df['is_correct'].sum() / len(results_df)

    # 保存 CSV
    results_df.to_csv(f'./Matched_results/Test2_matched_results_{file_prefix}.csv', index=False)
    print(f"[{file_prefix}] 匹配结果已保存，准确率: {accuracy * 100:.2f}%")
    return accuracy

# 计算准确率
df_list = [df_1,df_3,df_5,df_6,df_7,df_8]
df_list_name = ["df_1","df_3","df_5","df_6","df_7","df_8"]
for i in range(5):
    accuracy = fuzzy_match_test_to_primary(df_list[i], primary_names, file_prefix=str(df_list_name[i]))
    print(f"准确率: {accuracy * 100:.2f}%")

    ID                   NAME
0   36         AERO-CARIBBEAN
1  173            AVIA IMPORT
2  306  NATIONAL BANK OF CUBA
3  540                  COIBA
4  552                 CRYMSA
    ID                       NAME TYPE
0   36     AEROCARIBBEAN AIRLINES    C
1  173  ANGLO-CARIBBEAN CO., LTD.    C
2  306     BANCO NACIONAL DE CUBA    C
3  424         BOUTIQUE LA MAISON    C
4  475               CASA DE CUBA    C
      ID                                               NAME
0  16463                            PADIERNAPENALuisOrlando
1  18750  INSURANCECOMPANYSBERBANKINSURANCELIMITEDLIABIL...
2  52600                                 PORTEXTRADELIMITED
3  48766                       MAKAROVSergeiVyacheslavovich
4  19889                                    ISILSAUDIARABIA


匹配中 df_1: 100%|██████████| 1000/1000 [00:30<00:00, 32.79it/s]


[df_1] 匹配结果已保存，准确率: 99.70%
准确率: 99.70%


匹配中 df_3: 100%|██████████| 1000/1000 [00:29<00:00, 33.42it/s]


[df_3] 匹配结果已保存，准确率: 99.50%
准确率: 99.50%


...

[df_5] 匹配结果已保存，准确率: 91.00%
准确率: 91.00%


匹配中 df_6: 100%|██████████| 1000/1000 [00:42<00:00, 23.52it/s]


[df_6] 匹配结果已保存，准确率: 98.90%
准确率: 98.90%


匹配中 df_7: 100%|██████████| 1000/1000 [00:48<00:00, 20.81it/s]

[df_7] 匹配结果已保存，准确率: 89.10%
准确率: 89.10%


In [7]:
import pandas as pd
from rapidfuzz import process, fuzz
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm 
import re
import wordninja # 导入 tqdm

# 加载数据
alternate_names = pd.read_csv('alternate.csv', nrows=1000)
primary_names = pd.read_csv('primary.csv')
test_names = df_1

# 查看数据
print(alternate_names.head())
print(primary_names.head())
print(test_names.head())

# 将 primary_names 的 NAME 字段转换为列表，方便快速匹配
primary_names_list = primary_names['NAME'].tolist()
primary_ids = primary_names['ID'].tolist()

def normalize_and_sort_words(name):
    clean = re.sub(r'[^a-zA-Z0-9]', '', name)
    # Step 2: 小写（便于 wordninja 识别）再切分
    segments = wordninja.split(clean.lower())
    # Step 3: 恢复大写首字母（可选）
    return ' '.join(segments)

def recover_and_segment(name):
    if pd.isna(name):  # 处理缺失值
        return ''
    clean = re.sub(r'[^a-zA-Z0-9]', '', name)  # 去掉所有非字母数字字符
    segments = wordninja.split(clean.lower())  # 分词
    return ' '.join([w.upper() for w in segments])  # 全大写输出

# 匹配函数
def match_test_variant(test_row):
    # test_id = test_row['ID']
    test_variant_split = recover_and_segment(test_row['NAME'])
    test_variant = test_row['NAME']

    
    # 使用 rapidfuzz 找到最匹配的 NAME
    match1, score1, _ = process.extractOne(
        test_variant, 
        primary_names_list,
        processor=lambda x: re.sub(r'[^a-z0-9]', '', x.lower()),
        scorer=fuzz.WRatio  # 使用 token_sort_ratio 提高匹配精度
    )

    match2, score2, _ = process.extractOne(
        test_variant_split, 
        primary_names_list,
        processor=lambda x: re.sub(r'[^a-z0-9 ]', '', x.lower()),
        scorer=fuzz.token_sort_ratio  # 使用 token_sort_ratio 提高匹配精度
    )

    if score1 > score2:
        match = match1
        score = score1
    else:
        match = match2
        score = score2
    # 获取匹配的 NAME 对应的 ID
    matched_index = primary_names_list.index(match)
    matched_id = primary_ids[matched_index]

    # 返回匹配结果
    return matched_id, score

# 并行匹配
def fuzzy_match_test_to_primary(test_names, primary_names):
    correct_matches = 0  # 记录正确匹配的数量
    total_tests = len(test_names)  # 总测试数量

    # 使用多线程并行匹配
    with ThreadPoolExecutor() as executor:
        # 提交任务
        futures = [
            executor.submit(match_test_variant, test_row)
            for test_row in test_names.to_dict('records')
        ]

        # 使用 tqdm 包装 futures，显示进度条
        for future in tqdm(as_completed(futures), total=total_tests, desc="匹配进度"):
            matched_id, score = future.result()
            if score > 80:
                correct_matches += 1

    # 计算准确率
    accuracy = correct_matches / total_tests
    return accuracy

# 计算准确率
# df_list = [df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8]
# for i in range(7):
#     accuracy = fuzzy_match_test_to_primary(df_list[i], primary_names)
#     print(f"准确率: {accuracy * 100:.2f}%")

accuracy = fuzzy_match_test_to_primary(df_8, primary_names)
print(f"准确率: {accuracy * 100:.2f}%")

    ID                   NAME
0   36         AERO-CARIBBEAN
1  173            AVIA IMPORT
2  306  NATIONAL BANK OF CUBA
3  540                  COIBA
4  552                 CRYMSA
    ID                       NAME TYPE
0   36     AEROCARIBBEAN AIRLINES    C
1  173  ANGLO-CARIBBEAN CO., LTD.    C
2  306     BANCO NACIONAL DE CUBA    C
3  424         BOUTIQUE LA MAISON    C
4  475               CASA DE CUBA    C
      ID                                               NAME
0  16463                            PADIERNAPENALuisOrlando
1  18750  INSURANCECOMPANYSBERBANKINSURANCELIMITEDLIABIL...
2  52600                                 PORTEXTRADELIMITED
3  48766                       MAKAROVSergeiVyacheslavovich
4  19889                                    ISILSAUDIARABIA


匹配进度: 100%|██████████| 1000/1000 [00:50<00:00, 19.83it/s]

准确率: 10.20%


In [8]:
import re
import wordninja
from rapidfuzz import fuzz

# ------------------ 文本预处理 ------------------
def normalize(text):
    # 全大写，去除标点和多余空格
    text = text.upper()
    text = re.sub(r'[^A-Z0-9]', '', text)
    return ' '.join(wordninja.split(text))

# ------------------ 匹配并评分 ------------------
def compare_pairs(name1, name2):
    raw1, raw2 = name1.strip(), name2.strip()
    norm1, norm2 = normalize(raw1), normalize(raw2)

    score_raw = fuzz.WRatio(raw1, raw2)
    score_norm = fuzz.token_sort_ratio(norm1, norm2)
    score_partial = fuzz.partial_ratio(norm1, norm2)

    best_score = max(score_raw, score_norm, score_partial)

    print("原始字符串:")
    print(f"  A: {raw1}")
    print(f"  B: {raw2}")
    print("预处理后:")
    print(f"  A: {norm1}")
    print(f"  B: {norm2}")
    print("匹配分数:")
    print(f"  WRatio:             {score_raw}")
    print(f"  token_sort_ratio:   {score_norm}")
    print(f"  partial_ratio:      {score_partial}")
    print(f"✅ 最佳匹配得分:        {best_score}")
    print("=" * 60)

# ------------------ 样本对 ------------------
pairs = [
    ("TRADINGLIMITEDGENERALALCARDINAL", "ALCARDINAL GENERAL TRiADING LIMITED"),
    ("PADIERNAPENALuisOrlando", "Luisg PENA, PADIERNAM Orlando"),
    ("G O L D E N S T A R C O", "GOLDEN TAR CO"),
    ("S H A N G H A I S E A X U A N W U L T D F R E I G H T Y O U B I C O", "XUANWU YOUBI SEA FREI`HT SHANGHAI CO LTD"),
    ("HITTA, Sidan", "HITTA, Sidan Ag"),
    ("PREM INVESTMENT GROUP SAL (OFF-SHORE)", "PREMIER INVESTMENT GROUP SAL (OFF-SHORE)"),
    ("SHELESTENKO, Hennadiy O.", "SHELESTENKO, Hennadiy Oleksandrovych")
]

# ------------------ 执行匹配 ------------------
for a, b in pairs:
    compare_pairs(a, b)


原始字符串:
  A: TRADINGLIMITEDGENERALALCARDINAL
  B: ALCARDINAL GENERAL TRiADING LIMITED
预处理后:
  A: TRADING LIMITED GENERAL AL CARDINAL
  B: AL CARDINAL GENERAL TRIAD ING LIMITED
匹配分数:
  WRatio:             51.515151515151516
  token_sort_ratio:   88.88888888888889
  partial_ratio:      57.692307692307686
✅ 最佳匹配得分:        88.88888888888889
原始字符串:
  A: PADIERNAPENALuisOrlando
  B: Luisg PENA, PADIERNAM Orlando
预处理后:
  A: PA DIE RNA PENA LUIS ORLANDO
  B: LUIS GP E NAPA DIE RNA M ORLANDO
匹配分数:
  WRatio:             57.692307692307686
  token_sort_ratio:   66.66666666666667
  partial_ratio:      79.16666666666666
✅ 最佳匹配得分:        79.16666666666666
原始字符串:
  A: G O L D E N S T A R C O
  B: GOLDEN TAR CO
预处理后:
  A: GOLDEN STAR CO
  B: GOLDEN T ARCO
匹配分数:
  WRatio:             72.22222222222221
  token_sort_ratio:   81.4814814814815
  partial_ratio:      84.61538461538461
✅ 最佳匹配得分:        84.61538461538461
原始字符串:
  A: S H A N G H A I S E A X U A N W U L T D F R E I G H T Y O U B I C O
  B: XUANWU